# ScanImage Metadata Reference

Test datasets: `s1/mbospace/foconnell/lbm/2025-12-18_metadata-test`

This notebook validates metadata detection for:
1. LBM single vs dual channel detection
2. Piezo stack with/without frame averaging detection
3. Single z-plane acquisition

In [56]:
from pathlib import Path
import mbo_utilities as mbo

base_path = Path(r"\\rbo-s1\mbospace\foconnell\lbm\2025-12-18_metadata-test")

## 1. LBM Single vs Dual Channel Detection

Color channels in LBM are detected by counting unique AI sources in `si.hScan2D.virtualChannelSettings__N.source`

In [57]:
def get_ai_sources(metadata):
    """extract unique AI sources from virtualChannelSettings"""
    scan2d = metadata.get('si', {}).get('hScan2D', {})
    sources = {}
    for key, val in scan2d.items():
        if key.startswith('virtualChannelSettings__') and isinstance(val, dict):
            src = val.get('source')
            if src:
                if src not in sources:
                    sources[src] = []
                sources[src].append(int(key.split('__')[1]))
    return sources

In [58]:
# single channel LBM
single_file = base_path / "lbm" / "lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif"
single_arr = mbo.imread(single_file)
single_sources = get_ai_sources(single_arr.metadata)

print(f"Single channel LBM: {single_file.name}")
print(f"  Shape: {single_arr.shape}")
print(f"  AI sources: {list(single_sources.keys())}")
print(f"  num_color_channels: {len(single_sources)}")

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Single channel LBM: lbm_AI0-14ch_2mROIs_224px_RGG_uniform-sampling-ON_Avg1_00001.tif
  Shape: (100, 14, 448, 448)
  AI sources: ['AI0']
  num_color_channels: 1


In [59]:
# dual channel LBM
dual_file = base_path / "dual-channel-lbm" / "lbm_AI0-14ch_AI1-3ch_2mROIs_224px_RGG_uniform-sampling-ON_00001.tif"
dual_arr = mbo.imread(dual_file)
dual_sources = get_ai_sources(dual_arr.metadata)

print(f"Dual channel LBM: {dual_file.name}")
print(f"  Shape: {dual_arr.shape}")
print(f"  AI sources: {list(dual_sources.keys())}")
print(f"  num_color_channels: {len(dual_sources)}")

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Dual channel LBM: lbm_AI0-14ch_AI1-3ch_2mROIs_224px_RGG_uniform-sampling-ON_00001.tif
  Shape: (10, 17, 448, 448)
  AI sources: ['AI0', 'AI1']
  num_color_channels: 2


## 2. Piezo Stack: framesPerSlice Detection

Use `si.hStackManager.framesPerSlice`, NOT `si.hScan2D.logFramesPerSlice`

In [60]:
# file with multiple frames per slice (no averaging)
stack_file = Path(r"\\rbo-c2\D\C2_DATA\2025-12-02\StyletD_2CH_Stack_00002.tif")
stack_arr = mbo.imread(stack_file)
meta = stack_arr.metadata
si = meta.get('si', {})
stack_mgr = si.get('hStackManager', {})
scan2d = si.get('hScan2D', {})

print(f"Piezo stack: {stack_file.name}")
print(f"  Shape: {stack_arr.shape}")
print(f"  si.hStackManager.framesPerSlice: {stack_mgr.get('framesPerSlice')}")
print(f"  si.hScan2D.logFramesPerSlice: {scan2d.get('logFramesPerSlice')}")
print(f"  si.hStackManager.numSlices: {stack_mgr.get('numSlices')}")
print(f"  si.hScan2D.logAverageFactor: {scan2d.get('logAverageFactor')}")

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Piezo stack: StyletD_2CH_Stack_00002.tif
  Shape: (170, 2, 672, 669)
  si.hStackManager.framesPerSlice: 10
  si.hScan2D.logFramesPerSlice: None
  si.hStackManager.numSlices: 17
  si.hScan2D.logAverageFactor: 1


In [63]:
# total_frames / numSlices should equal framesPerSlice
total_frames = stack_arr.shape[0]
num_slices = stack_mgr.get('numSlices', 1)
frames_per_slice = stack_mgr.get('framesPerSlice', 1)

print(f"Verification:")
print(f"  total_frames / numSlices = {total_frames} / {num_slices} = {total_frames // num_slices}")
print(f"  framesPerSlice from metadata: {frames_per_slice}")
print(f"  Match: {total_frames // num_slices == frames_per_slice}")

Verification:
  total_frames / numSlices = 170 / 17 = 10
  framesPerSlice from metadata: 10
  Match: True


## 3. Frame Averaging Detection

`si.hScan2D.logAverageFactor > 1` indicates frames were averaged before saving

In [64]:
# averaged stack
avg_file = base_path / "stack_averaging" / "stack_AI0-1ch_3xZoom_224px_RGG_uniform-sampling-ON_17volume_11slices_10frames-per-slice_Avg10_00001.tif"
avg_arr = mbo.imread(avg_file)
avg_meta = avg_arr.metadata
avg_scan2d = avg_meta.get('si', {}).get('hScan2D', {})

print(f"Averaged stack: {avg_file.name}")
print(f"  Shape: {avg_arr.shape}")
print(f"  logAverageFactor: {avg_scan2d.get('logAverageFactor')}")

Counting frames:   0%|          | 0/1 [00:00<?, ?it/s]

Averaged stack: stack_AI0-1ch_3xZoom_224px_RGG_uniform-sampling-ON_17volume_11slices_10frames-per-slice_Avg10_00001.tif
  Shape: (187, 1, 224, 224)
  logAverageFactor: 10


## Summary

| Parameter | ScanImage Path | Notes |
|-----------|----------------|-------|
| LBM detection | `len(si.hChannels.channelSave) > 2` | LBM uses channels as z-planes |
| LBM color channels | `si.hScan2D.virtualChannelSettings__N.source` | AI0=1ch, AI0+AI1=2ch |
| Piezo detection | `si.hStackManager.enable` | True = piezo enabled |
| Z-planes | `si.hStackManager.numSlices` | |
| Z-step | `si.hStackManager.stackZStepSize` | microns |
| Frames per slice | `si.hStackManager.framesPerSlice` | NOT logFramesPerSlice |
| Averaging | `si.hScan2D.logAverageFactor` | >1 = averaged |